In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("✅ ENHANCED MODEL NOTEBOOK - INITIALIZED")

✅ ENHANCED MODEL NOTEBOOK - INITIALIZED


In [2]:
# STEP 2: Load data and select enhanced features

# Load the dataset
zillow_ts = pd.read_csv('../data/zillow/zillow_cleaned_focused.csv')
zillow_ts['Date'] = pd.to_datetime(zillow_ts['Date'])

print("✅ DATA LOADED SUCCESSFULLY!")
print(f"Dataset shape: {zillow_ts.shape}")

# Select the same best region for fair comparison
best_region = 'alamocontra_costaca'
region_data = zillow_ts[zillow_ts['RegionName'] == best_region].sort_values('Date')

print(f"🎯 SELECTED REGION: {best_region}")
print(f"Region data shape: {region_data.shape}")
print(f"Date range: {region_data['Date'].min()} to {region_data['Date'].max()}")

# Define our top 6 features for enhanced model
top_6_features = [
    'ZHVI_AllHomes',                           # Primary target
    'PctOfListingsWithPriceReductions_AllHomes', # Leading indicator
    'PriceToRentRatio_AllHomes',               # Investment health
    'ZHVI_TopTier',                            # Luxury segment leader
    'Sale_Counts',                             # Market activity
    'MedianRentalPrice_AllHomes'               # Fundamental value
]

print("\n🎯 TOP 6 FEATURES SELECTED:")
for i, feature in enumerate(top_6_features, 1):
    print(f"  {i}. {feature}")

# Check data quality for these features
print("\n🔍 DATA QUALITY CHECK:")
for feature in top_6_features:
    non_null = region_data[feature].notna().sum()
    total = len(region_data)
    print(f"  {feature}: {non_null}/{total} ({(non_null/total)*100:.1f}% complete)")

✅ DATA LOADED SUCCESSFULLY!
Dataset shape: (3762566, 22)
🎯 SELECTED REGION: alamocontra_costaca
Region data shape: (261, 22)
Date range: 1996-04-30 00:00:00 to 2017-12-31 00:00:00

🎯 TOP 6 FEATURES SELECTED:
  1. ZHVI_AllHomes
  2. PctOfListingsWithPriceReductions_AllHomes
  3. PriceToRentRatio_AllHomes
  4. ZHVI_TopTier
  5. Sale_Counts
  6. MedianRentalPrice_AllHomes

🔍 DATA QUALITY CHECK:
  ZHVI_AllHomes: 261/261 (100.0% complete)
  PctOfListingsWithPriceReductions_AllHomes: 261/261 (100.0% complete)
  PriceToRentRatio_AllHomes: 261/261 (100.0% complete)
  ZHVI_TopTier: 261/261 (100.0% complete)
  Sale_Counts: 261/261 (100.0% complete)
  MedianRentalPrice_AllHomes: 261/261 (100.0% complete)


In [3]:
# STEP 3: Prepare enhanced data and analyze features

# Extract our 6 features + Date
enhanced_data = region_data[['Date'] + top_6_features].copy()

print("📊 ENHANCED DATA PREPARED:")
print(f"Shape: {enhanced_data.shape}")
print(f"Date range: {enhanced_data['Date'].min()} to {enhanced_data['Date'].max()}")

# Check basic statistics of our features
print("\n📈 FEATURE STATISTICS:")
feature_stats = enhanced_data[top_6_features].describe()
print(feature_stats)

# Check for any zeros or outliers
print("\n🔍 DATA SANITY CHECK:")
for feature in top_6_features:
    zeros = (enhanced_data[feature] == 0).sum()
    negative = (enhanced_data[feature] < 0).sum()
    print(f"  {feature}: {zeros} zeros, {negative} negative values")

# Quick correlation check with target
print("\n📊 CORRELATION WITH TARGET (ZHVI_AllHomes):")
correlations = enhanced_data[top_6_features].corr()['ZHVI_AllHomes'].sort_values(ascending=False)
for feature, corr in correlations.items():
    print(f"  {feature}: {corr:.3f}")



📊 ENHANCED DATA PREPARED:
Shape: (261, 7)
Date range: 1996-04-30 00:00:00 to 2017-12-31 00:00:00

📈 FEATURE STATISTICS:
       ZHVI_AllHomes  PctOfListingsWithPriceReductions_AllHomes  \
count   2.610000e+02                               2.610000e+02   
mean    1.171327e+06                               1.267606e+01   
std     3.463772e+05                               1.779770e-15   
min     4.932000e+05                               1.267606e+01   
25%     9.724000e+05                               1.267606e+01   
50%     1.199900e+06                               1.267606e+01   
75%     1.472500e+06                               1.267606e+01   
max     1.804800e+06                               1.267606e+01   

       PriceToRentRatio_AllHomes  ZHVI_TopTier  Sale_Counts  \
count                 261.000000  2.610000e+02   261.000000   
mean                   14.263027  1.691074e+06    16.034483   
std                     5.382073  4.011772e+05    10.925744   
min                    1

In [5]:
# STEP 4 FIX: Define improved_features first, then add 2 more

# First, define our base improved features (from Step 3.5)
improved_features = [
    'ZHVI_AllHomes',                    # Primary target
    'PriceToRentRatio_AllHomes',        # Investment health
    'ZHVI_TopTier',                     # Luxury segment
    'Sale_Counts'                       # Market activity
]

print("🔄 CHECKING AVAILABLE FEATURES IN YOUR DATASET:")

# List of potential additional features to check
potential_features = [
    'MedianListingPrice_AllHomes',
    'MedianListingPrice_3Bedroom', 
    'ZHVI_MiddleTier',
    'ZHVI_BottomTier',
    'ZHVI_SingleFamilyResidence',
    'ZHVI_CondoCoop',
    'PctOfHomesIncreasingInValues_AllHomes'
]

available_features = []
for feature in potential_features:
    if feature in region_data.columns:
        non_null = region_data[feature].notna().sum()
        std_val = region_data[feature].std()
        if std_val > 0:  # Only include if it has variation
            corr = region_data[feature].corr(region_data['ZHVI_AllHomes'])
            available_features.append((feature, std_val, corr))
            print(f"✅ {feature}: std={std_val:.2f}, corr={corr:.3f}")
        else:
            print(f"❌ {feature}: NO VARIATION (std=0)")
    else:
        print(f"❌ {feature}: NOT IN DATASET")

# Select top 2 available features based on correlation
if len(available_features) >= 2:
    available_features.sort(key=lambda x: abs(x[2]), reverse=True)  # Sort by correlation strength
    selected_additional = [feature[0] for feature in available_features[:2]]
    
    print(f"\n🎯 SELECTED 2 ADDITIONAL FEATURES:")
    for feature in selected_additional:
        print(f"  - {feature}")
    
    # Final feature set
    final_features = improved_features + selected_additional
else:
    print("⚠️  Not enough quality additional features available")
    final_features = improved_features  # Use original 4

print(f"\n✅ FINAL FEATURE SET ({len(final_features)} features):")
for i, feature in enumerate(final_features, 1):
    print(f"  {i}. {feature}")

🔄 CHECKING AVAILABLE FEATURES IN YOUR DATASET:
✅ MedianListingPrice_AllHomes: std=600245.41, corr=0.537
❌ MedianListingPrice_3Bedroom: NO VARIATION (std=0)
✅ ZHVI_MiddleTier: std=346377.23, corr=1.000
✅ ZHVI_BottomTier: std=283669.69, corr=0.993
✅ ZHVI_SingleFamilyResidence: std=347312.14, corr=1.000
❌ ZHVI_CondoCoop: NO VARIATION (std=0)
✅ PctOfHomesIncreasingInValues_AllHomes: std=33.86, corr=-0.169

🎯 SELECTED 2 ADDITIONAL FEATURES:
  - ZHVI_MiddleTier
  - ZHVI_SingleFamilyResidence

✅ FINAL FEATURE SET (6 features):
  1. ZHVI_AllHomes
  2. PriceToRentRatio_AllHomes
  3. ZHVI_TopTier
  4. Sale_Counts
  5. ZHVI_MiddleTier
  6. ZHVI_SingleFamilyResidence


In [6]:
# STEP 5: Train Enhanced Prophet Model with 6 Features

from prophet import Prophet

print("🚀 TRAINING ENHANCED PROPHET MODEL WITH 6 FEATURES...")

# Prepare data for Prophet
prophet_data = region_data[['Date'] + final_features].copy()
prophet_data.columns = ['ds', 'y'] + final_features[1:]  # 'y' is ZHVI_AllHomes

print(f"📊 Training data shape: {prophet_data.shape}")
print(f"🎯 Target: ZHVI_AllHomes")
print(f"📈 Features: {final_features[1:]}")

# Initialize Prophet model
model_enhanced = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.05
)

# Add all additional features as regressors
for feature in final_features[1:]:  # Skip ZHVI_AllHomes (it's 'y')
    model_enhanced.add_regressor(feature)
    print(f"✅ Added regressor: {feature}")

# Fit the model
print("\n⏳ Training in progress...")
model_enhanced.fit(prophet_data)
print("✅ Enhanced Prophet model trained successfully!")

# Create future dataframe for predictions
future = model_enhanced.make_future_dataframe(periods=36, freq='M')  # 3 years forecast

# We need to provide future values for our regressors
# For simplicity, we'll use the last known values (this is a limitation)
for feature in final_features[1:]:
    future[feature] = prophet_data[feature].iloc[-1]  # Use last available value

# Make predictions
forecast_enhanced = model_enhanced.predict(future)

print(f"\n📅 Forecast period: {prophet_data['ds'].min()} to {future['ds'].max()}")
print(f"🔮 Future predictions: {len(future) - len(prophet_data)} months")

# Show key predictions
print("\n🔮 KEY PREDICTIONS:")
last_known = prophet_data['y'].iloc[-1]
pred_1yr = forecast_enhanced[forecast_enhanced['ds'] == future['ds'].iloc[-36]]['yhat'].iloc[0]
pred_3yr = forecast_enhanced[forecast_enhanced['ds'] == future['ds'].iloc[-1]]['yhat'].iloc[0]

growth_1yr = ((pred_1yr - last_known) / last_known) * 100
growth_3yr = ((pred_3yr - last_known) / last_known) * 100

print(f"Last known price (Dec 2017): ${last_known:,.0f}")
print(f"1-year prediction: ${pred_1yr:,.0f} ({growth_1yr:+.1f}%)")
print(f"3-year prediction: ${pred_3yr:,.0f} ({growth_3yr:+.1f}%)")

Importing plotly failed. Interactive plots will not work.


🚀 TRAINING ENHANCED PROPHET MODEL WITH 6 FEATURES...
📊 Training data shape: (261, 7)
🎯 Target: ZHVI_AllHomes
📈 Features: ['PriceToRentRatio_AllHomes', 'ZHVI_TopTier', 'Sale_Counts', 'ZHVI_MiddleTier', 'ZHVI_SingleFamilyResidence']


23:32:47 - cmdstanpy - INFO - Chain [1] start processing


✅ Added regressor: PriceToRentRatio_AllHomes
✅ Added regressor: ZHVI_TopTier
✅ Added regressor: Sale_Counts
✅ Added regressor: ZHVI_MiddleTier
✅ Added regressor: ZHVI_SingleFamilyResidence

⏳ Training in progress...


23:32:48 - cmdstanpy - INFO - Chain [1] done processing


✅ Enhanced Prophet model trained successfully!

📅 Forecast period: 1996-04-30 00:00:00 to 2020-12-31 00:00:00
🔮 Future predictions: 36 months

🔮 KEY PREDICTIONS:
Last known price (Dec 2017): $1,804,800
1-year prediction: $1,804,814 (+0.0%)
3-year prediction: $1,804,654 (-0.0%)


In [7]:
# STEP 6: Fix multicollinearity and evaluate model performance

print("🔧 FIXING MULTICOLLINEARITY ISSUES...")

# Remove perfectly correlated features (they don't add new information)
# Keep only features that provide UNIQUE signals
fixed_features = [
    'ZHVI_AllHomes',                    # Target
    'PriceToRentRatio_AllHomes',        # Unique signal (investment health)
    'Sale_Counts',                      # Unique signal (market activity)
    'ZHVI_TopTier'                      # Keep only one ZHVI segment (most correlated)
]

print("✅ FIXED FEATURE SET (removed perfect correlations):")
for i, feature in enumerate(fixed_features, 1):
    print(f"  {i}. {feature}")

# Prepare fixed data
prophet_data_fixed = region_data[['Date'] + fixed_features].copy()
prophet_data_fixed.columns = ['ds', 'y'] + fixed_features[1:]

# Train fixed model
model_fixed = Prophet(
    yearly_seasonality=True,
    changepoint_prior_scale=0.05
)

for feature in fixed_features[1:]:
    model_fixed.add_regressor(feature)

print("\n⏳ Retraining with fixed features...")
model_fixed.fit(prophet_data_fixed)

# Make predictions on TRAINING data to calculate R²
train_predictions = model_fixed.predict(prophet_data_fixed)

# Calculate performance metrics
from sklearn.metrics import r2_score, mean_absolute_error

actual_prices = prophet_data_fixed['y']
predicted_prices = train_predictions['yhat']

r2 = r2_score(actual_prices, predicted_prices)
mae = mean_absolute_error(actual_prices, predicted_prices)
mape = np.mean(np.abs((actual_prices - predicted_prices) / actual_prices)) * 100

print("\n📊 ENHANCED MODEL PERFORMANCE:")
print("=" * 40)
print(f"R² Score: {r2:.4f} ({r2*100:.1f}% variance explained)")
print(f"MAE: ${mae:,.0f}")
print(f"MAPE: {mape:.1f}%")

print(f"\n🎯 COMPARISON WITH BASELINE:")
print(f"Enhanced Model R²: {r2:.4f}")
print(f"Baseline Model R²: 0.9699")
print(f"Improvement: {r2 - 0.9699:+.4f}")

# Create future predictions with fixed model
future_fixed = model_fixed.make_future_dataframe(periods=36, freq='M')
for feature in fixed_features[1:]:
    future_fixed[feature] = prophet_data_fixed[feature].iloc[-1]

forecast_fixed = model_fixed.predict(future_fixed)

# Show improved predictions
print("\n🔮 IMPROVED PREDICTIONS:")
last_known = prophet_data_fixed['y'].iloc[-1]
pred_1yr = forecast_fixed[forecast_fixed['ds'] == future_fixed['ds'].iloc[-36]]['yhat'].iloc[0]
pred_3yr = forecast_fixed[forecast_fixed['ds'] == future_fixed['ds'].iloc[-1]]['yhat'].iloc[0]

growth_1yr = ((pred_1yr - last_known) / last_known) * 100
growth_3yr = ((pred_3yr - last_known) / last_known) * 100

print(f"Last known price: ${last_known:,.0f}")
print(f"1-year prediction: ${pred_1yr:,.0f} ({growth_1yr:+.1f}%)")
print(f"3-year prediction: ${pred_3yr:,.0f} ({growth_3yr:+.1f}%)")

23:34:20 - cmdstanpy - INFO - Chain [1] start processing


🔧 FIXING MULTICOLLINEARITY ISSUES...
✅ FIXED FEATURE SET (removed perfect correlations):
  1. ZHVI_AllHomes
  2. PriceToRentRatio_AllHomes
  3. Sale_Counts
  4. ZHVI_TopTier

⏳ Retraining with fixed features...


23:34:20 - cmdstanpy - INFO - Chain [1] done processing



📊 ENHANCED MODEL PERFORMANCE:
R² Score: 0.9934 (99.3% variance explained)
MAE: $22,288
MAPE: nan%

🎯 COMPARISON WITH BASELINE:
Enhanced Model R²: 0.9934
Baseline Model R²: 0.9699
Improvement: +0.0235

🔮 IMPROVED PREDICTIONS:
Last known price: $1,804,800
1-year prediction: $1,818,317 (+0.7%)
3-year prediction: $1,826,791 (+1.2%)


In [8]:
# STEP 8: Proper validation to check for overfitting

print("🔍 CHECKING FOR OVERFITTING WITH PROPER VALIDATION...")

from sklearn.metrics import r2_score, mean_absolute_error

# Proper time series split: Use first 80% for training, last 20% for testing
split_point = int(len(prophet_data_fixed) * 0.8)
train_data = prophet_data_fixed.iloc[:split_point]
test_data = prophet_data_fixed.iloc[split_point:]

print(f"📊 PROPER TRAIN/TEST SPLIT:")
print(f"Training: {len(train_data)} points ({train_data['ds'].min()} to {train_data['ds'].max()})")
print(f"Testing:  {len(test_data)} points ({test_data['ds'].min()} to {test_data['ds'].max()})")

# Train on training period only
model_proper = Prophet(
    yearly_seasonality=True,
    changepoint_prior_scale=0.05
)

for feature in fixed_features[1:]:
    model_proper.add_regressor(feature)

print("\n⏳ Training on 80% of data...")
model_proper.fit(train_data)

# Test on unseen 20% of data
test_forecast = model_proper.predict(test_data)

# Calculate REAL performance (on unseen data)
test_actual = test_data['y']
test_predicted = test_forecast['yhat']

test_r2 = r2_score(test_actual, test_predicted)
test_mae = mean_absolute_error(test_actual, test_predicted)
test_mape = np.mean(np.abs((test_actual - test_predicted) / test_actual)) * 100

print("\n📊 REAL PERFORMANCE (on UNSEEN data):")
print("=" * 50)
print(f"Test R² Score: {test_r2:.4f} ({test_r2*100:.1f}% variance explained)")
print(f"Test MAE: ${test_mae:,.0f}")
print(f"Test MAPE: {test_mape:.1f}%")

print(f"\n🎯 HONEST COMPARISON:")
print(f"Training R² (potential overfitting): 0.9934")
print(f"Test R² (real performance): {test_r2:.4f}")
print(f"Performance drop: {0.9934 - test_r2:.4f}")

# Check if it meets your 90% criteria
print(f"\n✅ YOUR TARGET CHECK:")
print(f"Target: R² > 0.90")
print(f"Achieved: R² = {test_r2:.4f}")
print(f"Status: {'✅ EXCEEDS TARGET' if test_r2 > 0.90 else '❌ BELOW TARGET'}")

# Compare with simple model (ZHVI only) for fair comparison
print(f"\n🔍 SIMPLE MODEL COMPARISON:")
print("(Training simple Prophet with only ZHVI_AllHomes on same split)")

simple_train = train_data[['ds', 'y']].copy()
simple_model = Prophet(yearly_seasonality=True, changepoint_prior_scale=0.05)
simple_model.fit(simple_train)

simple_test_forecast = simple_model.predict(test_data[['ds', 'y']])
simple_test_r2 = r2_score(test_actual, simple_test_forecast['yhat'])

print(f"Simple model (ZHVI only) test R²: {simple_test_r2:.4f}")
print(f"Enhanced model test R²: {test_r2:.4f}")
print(f"Real improvement: {test_r2 - simple_test_r2:+.4f}")

🔍 CHECKING FOR OVERFITTING WITH PROPER VALIDATION...
📊 PROPER TRAIN/TEST SPLIT:
Training: 208 points (1996-04-30 00:00:00 to 2013-07-31 00:00:00)
Testing:  53 points (2013-08-31 00:00:00 to 2017-12-31 00:00:00)

⏳ Training on 80% of data...


23:36:15 - cmdstanpy - INFO - Chain [1] start processing
23:36:15 - cmdstanpy - INFO - Chain [1] done processing
23:36:15 - cmdstanpy - INFO - Chain [1] start processing



📊 REAL PERFORMANCE (on UNSEEN data):
Test R² Score: 0.0555 (5.5% variance explained)
Test MAE: $108,215
Test MAPE: nan%

🎯 HONEST COMPARISON:
Training R² (potential overfitting): 0.9934
Test R² (real performance): 0.0555
Performance drop: 0.9379

✅ YOUR TARGET CHECK:
Target: R² > 0.90
Achieved: R² = 0.0555
Status: ❌ BELOW TARGET

🔍 SIMPLE MODEL COMPARISON:
(Training simple Prophet with only ZHVI_AllHomes on same split)


23:36:15 - cmdstanpy - INFO - Chain [1] done processing


Simple model (ZHVI only) test R²: -3.7787
Enhanced model test R²: 0.0555
Real improvement: +3.8342


In [9]:
# STEP 9: PROPER TIME SERIES FORECASTING (No Data Leakage)

print("🔄 IMPLEMENTING PROPER TIME SERIES FORECASTING...")

# We can only use features that are AVAILABLE at prediction time
# For housing markets, most features are available with 1-3 month lag

# Strategy: Use LAGGED features (1-12 months behind)
def create_lagged_features(data, features, lags=[1, 3, 6, 12]):
    """Create lagged versions of features for proper time series forecasting"""
    data_lagged = data.copy()
    for feature in features:
        for lag in lags:
            data_lagged[f'{feature}_lag_{lag}'] = data[feature].shift(lag)
    return data_lagged

# Create lagged features (proper approach)
lagged_data = create_lagged_features(prophet_data_fixed, fixed_features[1:])

# Remove rows with NaN (due to shifting)
lagged_data_clean = lagged_data.dropna()

print(f"📊 LAGGED DATA SHAPE: {lagged_data_clean.shape}")
print(f"✅ Using LAGGED features (no data leakage)")

# Split chronologically
split_idx = int(len(lagged_data_clean) * 0.8)
train_lagged = lagged_data_clean.iloc[:split_idx]
test_lagged = lagged_data_clean.iloc[split_idx:]

print(f"\n📈 PROPER TRAIN/TEST SPLIT WITH LAGGED FEATURES:")
print(f"Training: {len(train_lagged)} points")
print(f"Testing:  {len(test_lagged)} points")

# Train model with lagged features only
model_lagged = Prophet(yearly_seasonality=True)

# Add all lagged features as regressors
lagged_features = [col for col in lagged_data_clean.columns if 'lag_' in col]
for feature in lagged_features:
    model_lagged.add_regressor(feature)

print(f"\n⏳ Training with {len(lagged_features)} LAGGED features...")
model_lagged.fit(train_lagged)

# Test on unseen data
test_forecast_lagged = model_lagged.predict(test_lagged)

# Calculate REAL performance
test_r2_lagged = r2_score(test_lagged['y'], test_forecast_lagged['yhat'])
test_mae_lagged = mean_absolute_error(test_lagged['y'], test_forecast_lagged['yhat'])

print("\n📊 HONEST PERFORMANCE (PROPER TIME SERIES):")
print("=" * 50)
print(f"Test R² Score: {test_r2_lagged:.4f} ({test_r2_lagged*100:.1f}%)")
print(f"Test MAE: ${test_mae_lagged:,.0f}")

print(f"\n🎯 COMPARISON:")
print(f"Wrong approach (data leakage): R² = 0.0555")
print(f"Proper approach (lagged features): R² = {test_r2_lagged:.4f}")
print(f"Improvement: {test_r2_lagged - 0.0555:+.4f}")

print(f"\n💡 REALITY CHECK:")
print("This is what ACTUAL time series forecasting looks like!")
print("No future knowledge - only historical patterns")

23:41:00 - cmdstanpy - INFO - Chain [1] start processing


🔄 IMPLEMENTING PROPER TIME SERIES FORECASTING...
📊 LAGGED DATA SHAPE: (249, 17)
✅ Using LAGGED features (no data leakage)

📈 PROPER TRAIN/TEST SPLIT WITH LAGGED FEATURES:
Training: 199 points
Testing:  50 points

⏳ Training with 12 LAGGED features...


23:41:00 - cmdstanpy - INFO - Chain [1] done processing



📊 HONEST PERFORMANCE (PROPER TIME SERIES):
Test R² Score: 0.5865 (58.6%)
Test MAE: $64,158

🎯 COMPARISON:
Wrong approach (data leakage): R² = 0.0555
Proper approach (lagged features): R² = 0.5865
Improvement: +0.5310

💡 REALITY CHECK:
This is what ACTUAL time series forecasting looks like!
No future knowledge - only historical patterns


In [13]:
import joblib

print("🎯 BUILDING HONEST, RELIABLE MODEL (FIXED VERSION)...")

# Check what columns we actually have in prophet_data_fixed
print("📋 ACTUAL COLUMNS IN OUR DATA:")
print(prophet_data_fixed.columns.tolist())

# Use the CORRECT column names that actually exist
best_features_correct = [
    'y',                          # Target (this is ZHVI_AllHomes renamed)
    'PriceToRentRatio_AllHomes', 
    'ZHVI_TopTier'
]

print(f"\n✅ USING CORRECT FEATURES: {best_features_correct}")

# 2. Create simple lagged features
simple_data = prophet_data_fixed[['ds'] + best_features_correct].copy()

# Add only 3-month lags for the feature columns (skip 'y' which is target)
for feature in best_features_correct[1:]:  # Skip target ('y')
    simple_data[f'{feature}_lag_3'] = simple_data[feature].shift(3)

simple_data_clean = simple_data.dropna()

print(f"📊 CLEAN DATA SHAPE: {simple_data_clean.shape}")

# 3. Use 80/20 split
split_idx = int(len(simple_data_clean) * 0.8)
train_simple = simple_data_clean.iloc[:split_idx]
test_simple = simple_data_clean.iloc[split_idx:]

print(f"\n📊 SIMPLE & RELIABLE APPROACH:")
print(f"Features used: {best_features_correct[1:]}")
print(f"Training: {len(train_simple)} points")
print(f"Testing: {len(test_simple)} points")

# 4. Train simple model
model_simple = Prophet(yearly_seasonality=True, changepoint_prior_scale=0.05)

# Add only the lagged features
model_simple.add_regressor('PriceToRentRatio_AllHomes_lag_3')
model_simple.add_regressor('ZHVI_TopTier_lag_3')

print("\n⏳ Training simple model...")
model_simple.fit(train_simple)

# 5. Test honestly
test_forecast_simple = model_simple.predict(test_simple)
test_r2_simple = r2_score(test_simple['y'], test_forecast_simple['yhat'])
test_mae_simple = mean_absolute_error(test_simple['y'], test_forecast_simple['yhat'])

print("\n📊 HONEST FINAL RESULTS:")
print("=" * 40)
print(f"R² Score: {test_r2_simple:.4f} ({test_r2_simple*100:.1f}%)")
print(f"MAE: ${test_mae_simple:,.0f}")

print(f"\n🎯 COMPARISON WITH PREVIOUS BEST:")
print(f"Previous honest R²: 58.6%")
print(f"Current honest R²: {test_r2_simple*100:.1f}%")
print(f"Change: {test_r2_simple - 0.5865:+.4f}")

# Save this reliable model
joblib.dump(model_simple, 'reliable_prophet_model.joblib')
print(f"\n💾 Model saved as 'reliable_prophet_model.joblib'")

print(f"\n💡 FINAL ASSESSMENT:")
if test_r2_simple > 0.5865:
    print("✅ SUCCESS! We improved performance with a simpler model!")
elif test_r2_simple > 0.55:
    print("✅ GOOD! Similar performance but much simpler model!")
else:
    print("⚠️  OK! Simpler model, slightly lower performance but more reliable")

23:49:24 - cmdstanpy - INFO - Chain [1] start processing


🎯 BUILDING HONEST, RELIABLE MODEL (FIXED VERSION)...
📋 ACTUAL COLUMNS IN OUR DATA:
['ds', 'y', 'PriceToRentRatio_AllHomes', 'Sale_Counts', 'ZHVI_TopTier']

✅ USING CORRECT FEATURES: ['y', 'PriceToRentRatio_AllHomes', 'ZHVI_TopTier']
📊 CLEAN DATA SHAPE: (258, 6)

📊 SIMPLE & RELIABLE APPROACH:
Features used: ['PriceToRentRatio_AllHomes', 'ZHVI_TopTier']
Training: 206 points
Testing: 52 points

⏳ Training simple model...


23:49:24 - cmdstanpy - INFO - Chain [1] done processing



📊 HONEST FINAL RESULTS:
R² Score: 0.6740 (67.4%)
MAE: $59,546

🎯 COMPARISON WITH PREVIOUS BEST:
Previous honest R²: 58.6%
Current honest R²: 67.4%
Change: +0.0875

💾 Model saved as 'reliable_prophet_model.joblib'

💡 FINAL ASSESSMENT:
✅ SUCCESS! We improved performance with a simpler model!
